# Part 1: Data Preparation
Codes executed in `submit` folder

## Setup

Import required libraries

In [1]:
import pandas as pd
from pathlib import Path
from typing import Union

Read in data files

In [2]:
# Define directories for inputs and output
DIR_CURR = Path.cwd()
DIR_DATA = DIR_CURR.parent / 'data'

# Read in data as Pandas DataFrame
cases_df = pd.read_pickle(DIR_DATA / 'df_cases_200906.gzip')
labels_df = pd.read_pickle(DIR_DATA / 'df_label_200906.gzip')

## Data Exploration

Examine top 5 rows of cases data

In [3]:
cases_df.head()

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-3...,EN,20191104,Order form,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0...,EN,20190805,Contract Info Pack,False,Soon especially boy thousand traditional.,0.873038
2,003061226383,003061226383_41775424_other documents_978-1-38...,EN,20190805,other documents,True,Tax south say strategy hard between late.,0.991280
3,003061227777,003061227777_96463321_other documents_978-1-96...,EN,20191231,other documents,False,Yet arm six design.,0.607833
4,003061227781,003061227781_62201704_other documents_978-1-04...,EN,20191231,other documents,False,Plan camera actually run push rest.,0.909359


Examine data type of columns and check for missing values in cases data (No missing values for cases data)

In [4]:
cases_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 0 to 2332
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CaseId        2069 non-null   object 
 1   FileName      2069 non-null   object 
 2   Language      2069 non-null   object 
 3   StartDate     2069 non-null   object 
 4   DocumentType  2069 non-null   object 
 5   IsExecuted    2069 non-null   object 
 6   OcrText       2069 non-null   object 
 7   QualityScore  2069 non-null   float64
dtypes: float64(1), object(7)
memory usage: 145.5+ KB


Check for repetitions of CaseID in cases data

In [5]:
cases_df['CaseId'].value_counts()

003061193920    11
003061196422     9
003061212307     8
003061230613     8
003061214084     8
                ..
003061207506     1
003061189351     1
003061190485     1
003061209767     1
003061203233     1
Name: CaseId, Length: 1098, dtype: int64

Examine top 5 rows of labels data

In [6]:
labels_df.head()

,CaseId,label_1,label_2
0,003061226227,False,False
1,003061226383,True,False
2,003061227777,False,False
3,003061227781,False,False
4,003061227680,True,True


Examine data type of columns and check for missing values in labels data (7 missing values for columns `label_1` and `label_2` in labels data)

In [7]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098 entries, 0 to 1097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CaseId   1098 non-null   object
 1   label_1  1091 non-null   object
 2   label_2  1091 non-null   object
dtypes: object(3)
memory usage: 25.9+ KB


Check for repetitions of CaseID in labels data

In [8]:
labels_df['CaseId'].value_counts()

003061203233    1
003061219979    1
003061212818    1
003061197265    1
003061211748    1
               ..
003061197280    1
003061211816    1
003061225585    1
003061194017    1
003061198565    1
Name: CaseId, Length: 1098, dtype: int64

Check that all unique CaseIDs found in cases data can also be found in labels data (and vice-versa)

In [9]:
assert set(cases_df['CaseId'].tolist()) == set(labels_df['CaseId'].tolist())

## Data Processing

Get unique case IDs in sorted order

In [10]:
case_ids_unique = sorted(set(cases_df['CaseId'].tolist()))

Define function to convert labels to boolean or NaN (for missing values in labels data)

In [11]:
def convert_label(label: Union[str, float]) -> Union[bool, float]:
    if label == 'True':
        return True
    if label == 'False':
        return False
    # If label is NaN
    return label

Convert fields under columns `label_1` and `label_2` to boolean if there are no missing values

In [12]:
labels_df['label_1'] = labels_df['label_1'].apply(convert_label)
labels_df['label_2'] = labels_df['label_2'].apply(convert_label)

Create `dict` to store case IDs, filenames for valid and invalid filenames, and OCR results for valid contracts. Then convert the resulting `dict` to a Pandas `DataFrame`.

In [13]:
# Initialize `output_dict`
output_dict = {
    'CaseId': [],
    'ValidFileNames': [],
    'InvalidFileNames': [],
    'OcrText': []
}

# Slice `cases_df` to get relevant values for `output_dict`
for case_id in case_ids_unique:
    # Get subset of cases data for individual case ID
    cases_subset_df = cases_df[cases_df['CaseId'] == case_id]
    
    # Get further subset for invalid contracts
    # Note: Fields under `IsExecuted` column are strings rather than booleans
    invalid_df = cases_subset_df[
        (cases_subset_df['IsExecuted'] == 'False') |
        (cases_subset_df['QualityScore'] < 0.81)
    ]
    
    # Get subset for valid contracts
    valid_df = cases_subset_df[
        ~cases_subset_df.index.isin(invalid_df.index)
    ]

    # Append values accordingly in `output_dict`
    output_dict['CaseId'].append(case_id)
    output_dict['InvalidFileNames'].append(invalid_df['FileName'].tolist())
    output_dict['ValidFileNames'].append(valid_df['FileName'].tolist())
    output_dict['OcrText'].append(' '.join(valid_df['OcrText'].tolist()))

# Convert resulting dict to Pandas DataFrame
output_df = pd.DataFrame(output_dict)

Use `pd.DataFrame.merge` method to add `label_1` and `label_2` values corresponding to `CaseId`

In [14]:
output_df = output_df.merge(labels_df, on='CaseId')

Examine top 5 rows of final output

In [15]:
output_df.head()

,CaseId,ValidFileNames,InvalidFileNames,OcrText,label_1,label_2
0,003061189006,[003061189006_69176036_Order form_978-1-62414-...,[],As difficult behavior her myself help.,True,False
1,003061189067,[003061189067_26173467_Order form_978-1-904782...,[],Present can phone form.,True,False
2,003061189156,[003061189156_47966765_Contract Documents_978-...,[],Section science difference success wish it wide.,False,False
3,003061189229,[003061189229_21094545_other documents_978-1-8...,[003061189229_69115288_Master contract_978-0-1...,Accept sell leader herself if.,False,False
4,003061189242,[],[003061189242_91658807_other documents_978-0-7...,,False,False


Write output DataFrame to compressed pickle file

In [16]:
output_df.to_pickle(DIR_CURR / 'df_final.gzip')